In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

In [6]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../../data/train_merge_sub_room.csv'
test_path  = '../../data/test_merge_sub_room.csv'
codebook_path = '../../data/codebook.csv'
gdp_path = '../../data/gdp(2007~).csv'
interest_rate_path = '../../data/ interest_rate.csv'
edu_group_path = '../../data/education_group.csv'
lease_rate_path = '../../data/lease_rate.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
codebook = pd.read_csv(codebook_path)
gdp = pd.read_csv(gdp_path)
interest_rate = pd.read_csv(interest_rate_path)
edu_group = pd.read_csv(edu_group_path)
lease_rate = pd.read_csv(lease_rate_path, encoding='cp949')

In [7]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [4]:
concat.columns

Index(['시군구', '번지', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)',
       'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y', '단지신청일', 'target', '시', '구', '동',
       '지번주소', '동아파트명', '전용면적2', '방개수2', '화장실개수2', '공간개수', '유동승객수', '역까지_거리',
       '1차역세권', '2차역세권', 'X좌표', 'Y좌표', '정류장까지_거리', 'is_test'],
      dtype='object')

In [6]:
concat['계약년'] = concat['계약년월'].astype('str').map(lambda x : x[:4])
concat['계약월'] = concat['계약년월'].astype('str').map(lambda x : x[4:])

## GDP 병합

In [7]:
# 월에 따른 분기를 만드는 함수
def make_quarter(month):
    if month in ["01", "02", "03"]:
        return "1/4"
    elif month in ["04", "05", "06"]:
        return "2/4"
    elif month in ["07", "08", "09"]:
        return "3/4"
    elif month in ["10", "11", "12"]:
        return "4/4"
    else:
        return "NaN" 

In [8]:
concat["계약분기"] = concat["계약월"].apply(make_quarter)

In [9]:
# 계약년에 분기를 더해 분기(계약년) 생성
concat["분기"] = concat['계약년']+concat["계약분기"]

In [10]:
# 분기를 기준으로 병합 1128094 rows
new_concat = pd.merge(concat, gdp, on="분기", how="inner")
new_concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,X좌표,Y좌표,정류장까지_거리,is_test,계약년,계약월,계약분기,분기,국내총생산(명목GDP),경제성장률(실질GDP성장률)
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,127.057864,37.476564,73.917793,0,2017,12,4/4,20174/4,"474,957.90",2.9
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,127.057864,37.476564,73.917793,0,2017,12,4/4,20174/4,"474,957.90",2.9
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,127.057864,37.476564,73.917793,0,2017,12,4/4,20174/4,"474,957.90",2.9
3,서울특별시 강남구 개포동,652,개포우성3차,133.46,201710,13,1,1984,개포로 307,NaN,...,127.056890,37.483174,111.101602,0,2017,10,4/4,20174/4,"474,957.90",2.9
4,서울특별시 강남구 개포동,652,개포우성3차,161.00,201711,11,8,1984,개포로 307,NaN,...,127.056890,37.483174,111.101602,0,2017,11,4/4,20174/4,"474,957.90",2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,127.107073,37.619326,49.828552,1,2023,07,3/4,20233/4,"563,921.50",1.4
1128090,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,127.107073,37.619326,49.828552,1,2023,07,3/4,20233/4,"563,921.50",1.4
1128091,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,127.107073,37.619326,49.828552,1,2023,08,3/4,20233/4,"563,921.50",1.4
1128092,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,127.107073,37.619326,49.828552,1,2023,09,3/4,20233/4,"563,921.50",1.4


In [11]:
# 병합하기 전 원본 1128094 rows
concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,1차역세권,2차역세권,X좌표,Y좌표,정류장까지_거리,is_test,계약년,계약월,계약분기,분기
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,0,0,127.057864,37.476564,73.917793,0,2017,12,4/4,20174/4
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,0,0,127.057864,37.476564,73.917793,0,2017,12,4/4,20174/4
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,0,0,127.057864,37.476564,73.917793,0,2017,12,4/4,20174/4
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,0,0,127.057864,37.476564,73.917793,0,2018,01,1/4,20181/4
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,0,0,127.057864,37.476564,73.917793,0,2018,01,1/4,20181/4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,0,0,127.107073,37.619326,49.828552,1,2023,07,3/4,20233/4
9268,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,0,0,127.107073,37.619326,49.828552,1,2023,07,3/4,20233/4
9269,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,0,0,127.107073,37.619326,49.828552,1,2023,08,3/4,20233/4
9270,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,0,0,127.107073,37.619326,49.828552,1,2023,09,3/4,20233/4


In [12]:
new_concat[["국내총생산(명목GDP)", "경제성장률(실질GDP성장률)"]].isna().sum()

국내총생산(명목GDP)       0
경제성장률(실질GDP성장률)    0
dtype: int64

In [13]:
# 병합에 사용한 columns 삭제
del new_concat["분기"]
del new_concat["계약분기"]

## 기준금리 병합

In [14]:
interest_rate

,변경일자_년,변경일자_월일,기준금리
0,2023,01월 13일,3.50
1,2022,11월 24일,3.25
2,2022,10월 12일,3.00
3,2022,08월 25일,2.50
4,2022,07월 13일,2.25
5,2022,05월 26일,1.75
6,2022,04월 14일,1.50
7,2022,01월 14일,1.25
8,2021,11월 25일,1.00
9,2021,08월 26일,0.75


In [15]:
# 변경일자년월일 생성
interest_rate["일자년월일"] = interest_rate["변경일자_년"].astype(str) + interest_rate["변경일자_월일"].map(lambda x : x[:2] + x[4:6])


In [16]:
use_interest_rate = interest_rate.sort_values("일자년월일")[["일자년월일", "기준금리"]]

In [17]:
use_interest_rate

,일자년월일,기준금리
38,20060810,4.50
37,20070712,4.75
36,20070809,5.00
35,20080807,5.25
34,20081009,5.00
33,20081027,4.25
32,20081107,4.00
31,20081211,3.00
30,20090109,2.50
29,20090212,2.00


In [18]:
new_concat["일자년월일"] = new_concat["계약년월"].astype(str)+new_concat["계약일"].astype(str).str.zfill(2)

In [19]:
new_concat.sort_values(by="일자년월일", inplace=True)

In [20]:
new_concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,2차역세권,X좌표,Y좌표,정류장까지_거리,is_test,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),일자년월일
1079005,서울특별시 강서구 마곡동,429,벽산,59.980,200701,1,15,1999,양천로47길 118,NaN,...,0,126.825674,37.566753,41.796893,0,2007,01,"252,586.30",4.9,20070101
1082208,서울특별시 노원구 월계동,322-1,성원,59.940,200701,1,9,1995,마들로3길 37,NaN,...,1,127.061686,37.631583,192.711284,0,2007,01,"252,586.30",4.9,20070101
1083444,서울특별시 도봉구 도봉동,647,도봉파크빌2,84.354,200701,1,1,2002,마들로 835,NaN,...,1,127.043376,37.679657,218.273702,0,2007,01,"252,586.30",4.9,20070101
1090106,서울특별시 중랑구 상봉동,495,동부,84.990,200701,1,2,1999,신내로7가길 31,NaN,...,1,127.095466,37.603957,221.056734,0,2007,01,"252,586.30",4.9,20070101
1082788,서울특별시 노원구 중계동,445,염광,59.340,200701,1,8,1996,덕릉로79길 23,NaN,...,0,127.076783,37.647301,48.210662,0,2007,01,"252,586.30",4.9,20070101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125273,서울특별시 성북구 하월곡동,222,월곡두산위브,84.843,202309,25,13,2003,오패산로 46,20230926.0,...,1,127.038892,37.605201,135.628330,1,2023,09,"563,921.50",1.4,20230925
1122664,서울특별시 도봉구 쌍문동,59,한양2,35.100,202309,25,11,1988,해등로 231,NaN,...,0,127.027833,37.655624,163.035643,1,2023,09,"563,921.50",1.4,20230925
1126924,서울특별시 영등포구 문래동3가,93-1,문래건영,59.400,202309,25,11,1998,문래로 94-7,20230926.0,...,0,126.896353,37.516949,49.484311,1,2023,09,"563,921.50",1.4,20230925
1125941,서울특별시 송파구 방이동,50-10,동광팰리스,22.570,202309,26,14,2018,오금로11길 63-13,NaN,...,1,127.119104,37.514828,66.815991,1,2023,09,"563,921.50",1.4,20230926


In [21]:
new_concat = pd.merge(new_concat, use_interest_rate, on="일자년월일", how='left')

In [22]:
new_concat[~new_concat["기준금리"].isna()].sort_values(by="일자년월일")

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,X좌표,Y좌표,정류장까지_거리,is_test,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),일자년월일,기준금리
28839,서울특별시 성북구 정릉동,239,정릉풍림아이원,114.750,200707,12,13,2005,솔샘로25길 28,NaN,...,127.007432,37.617339,84.570851,0,2007,07,"275,865.80",5.5,20070712,4.75
28954,서울특별시 중구 회현동1가,100-12,이지-빌,55.610,200707,12,2,2006,퇴계로8길 65-3,NaN,...,126.980093,37.555737,162.566317,0,2007,07,"275,865.80",5.5,20070712,4.75
28955,서울특별시 노원구 하계동,273,장미,43.560,200707,12,3,1989,섬밭로 196,NaN,...,127.073649,37.637936,126.662827,0,2007,07,"275,865.80",5.5,20070712,4.75
28956,서울특별시 도봉구 창동,347,창동주공3단지,66.560,200707,12,3,1990,해등로 50,NaN,...,127.046171,37.651333,94.555414,0,2007,07,"275,865.80",5.5,20070712,4.75
28957,서울특별시 동대문구 회기동,347-7,회기,81.860,200707,12,4,2003,회기로 173-1,NaN,...,127.052247,37.594062,54.965610,0,2007,07,"275,865.80",5.5,20070712,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101759,서울특별시 도봉구 방학동,720-18,방학동삼성래미안1,118.381,202301,13,8,2002,마들로 646,NaN,...,127.034113,37.662595,150.062231,0,2023,01,"523,816.20",0.9,20230113,3.50
1101758,서울특별시 구로구 구로동,685-222,주공1,73.080,202301,13,9,1986,구일로4길 65,NaN,...,126.884354,37.493110,100.788928,0,2023,01,"523,816.20",0.9,20230113,3.50
1101757,서울특별시 노원구 상계동,995,한신4차,84.840,202301,13,12,1995,동일로230길 63,NaN,...,127.071604,37.665307,159.819527,0,2023,01,"523,816.20",0.9,20230113,3.50
1101765,서울특별시 성북구 장위동,65-239,라인캐슬,55.150,202301,13,5,2022,화랑로 213,NaN,...,127.050048,37.616430,91.595749,0,2023,01,"523,816.20",0.9,20230113,3.50


In [23]:
new_concat["기준금리"].fillna(method='ffill', inplace=True)

In [24]:
new_concat[new_concat["기준금리"].isna()].sort_values(by="일자년월일")

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,X좌표,Y좌표,정류장까지_거리,is_test,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),일자년월일,기준금리
0,서울특별시 강서구 마곡동,429,벽산,59.980,200701,1,15,1999,양천로47길 118,NaN,...,126.825674,37.566753,41.796893,0,2007,01,"252,586.30",4.9,20070101,NaN
13,서울특별시 성북구 하월곡동,225,래미안월곡,114.678,200701,1,18,2006,오패산로 90,NaN,...,127.037620,37.604146,19.799229,0,2007,01,"252,586.30",4.9,20070101,NaN
12,서울특별시 동작구 신대방동,395-62,삼성보라매옴니타워,164.760,200701,1,15,1997,보라매로5길 23,NaN,...,126.918701,37.494588,117.831731,0,2007,01,"252,586.30",4.9,20070101,NaN
10,서울특별시 동대문구 답십리동,42,동답한신,59.220,200701,1,7,1992,답십리로 210-30,NaN,...,127.054827,37.572769,185.018361,0,2007,01,"252,586.30",4.9,20070101,NaN
9,서울특별시 성북구 정릉동,1020,정릉우성,84.990,200701,1,10,2000,북악산로 813,NaN,...,127.015207,37.600602,61.417741,0,2007,01,"252,586.30",4.9,20070101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28715,서울특별시 성동구 응봉동,98,금호현대,79.800,200707,11,9,1990,독서당로 377,NaN,...,127.033501,37.553399,69.890608,0,2007,07,"275,865.80",5.5,20070711,NaN
28716,서울특별시 도봉구 창동,807,쌍용,135.000,200707,11,13,1997,마들로 551,NaN,...,127.044201,37.646931,109.874142,0,2007,07,"275,865.80",5.5,20070711,NaN
28717,서울특별시 노원구 상계동,1255,은빛1단지,59.920,200707,11,11,1998,동일로245길 162,NaN,...,127.071604,37.665307,159.819527,0,2007,07,"275,865.80",5.5,20070711,NaN
28708,서울특별시 노원구 공릉동,90,청솔,84.900,200707,11,6,2000,노원로1길 21,NaN,...,127.086477,37.623135,51.200377,0,2007,07,"275,865.80",5.5,20070711,NaN


In [25]:
new_concat["기준금리"].fillna(4.5, inplace=True)

In [26]:
del new_concat["일자년월일"]

In [27]:
new_concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,2차역세권,X좌표,Y좌표,정류장까지_거리,is_test,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리
0,서울특별시 강서구 마곡동,429,벽산,59.980,200701,1,15,1999,양천로47길 118,NaN,...,0,126.825674,37.566753,41.796893,0,2007,01,"252,586.30",4.9,4.5
1,서울특별시 노원구 월계동,322-1,성원,59.940,200701,1,9,1995,마들로3길 37,NaN,...,1,127.061686,37.631583,192.711284,0,2007,01,"252,586.30",4.9,4.5
2,서울특별시 도봉구 도봉동,647,도봉파크빌2,84.354,200701,1,1,2002,마들로 835,NaN,...,1,127.043376,37.679657,218.273702,0,2007,01,"252,586.30",4.9,4.5
3,서울특별시 중랑구 상봉동,495,동부,84.990,200701,1,2,1999,신내로7가길 31,NaN,...,1,127.095466,37.603957,221.056734,0,2007,01,"252,586.30",4.9,4.5
4,서울특별시 노원구 중계동,445,염광,59.340,200701,1,8,1996,덕릉로79길 23,NaN,...,0,127.076783,37.647301,48.210662,0,2007,01,"252,586.30",4.9,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 성북구 하월곡동,222,월곡두산위브,84.843,202309,25,13,2003,오패산로 46,20230926.0,...,1,127.038892,37.605201,135.628330,1,2023,09,"563,921.50",1.4,3.5
1128090,서울특별시 도봉구 쌍문동,59,한양2,35.100,202309,25,11,1988,해등로 231,NaN,...,0,127.027833,37.655624,163.035643,1,2023,09,"563,921.50",1.4,3.5
1128091,서울특별시 영등포구 문래동3가,93-1,문래건영,59.400,202309,25,11,1998,문래로 94-7,20230926.0,...,0,126.896353,37.516949,49.484311,1,2023,09,"563,921.50",1.4,3.5
1128092,서울특별시 송파구 방이동,50-10,동광팰리스,22.570,202309,26,14,2018,오금로11길 63-13,NaN,...,1,127.119104,37.514828,66.815991,1,2023,09,"563,921.50",1.4,3.5


In [28]:
save_train = new_concat[new_concat["is_test"]==0]
del save_train["is_test"]
save_train.to_csv('train_merge_0119_01(cp949).csv', index=False, encoding='cp949')
save_train.to_csv('train_merge_0119_01(UTF-8).csv', index=False, encoding='UTF-8')

In [29]:
save_train

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,1차역세권,2차역세권,X좌표,Y좌표,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리
0,서울특별시 강서구 마곡동,429,벽산,59.980,200701,1,15,1999,양천로47길 118,NaN,...,1,0,126.825674,37.566753,41.796893,2007,01,"252,586.30",4.9,4.5
1,서울특별시 노원구 월계동,322-1,성원,59.940,200701,1,9,1995,마들로3길 37,NaN,...,0,1,127.061686,37.631583,192.711284,2007,01,"252,586.30",4.9,4.5
2,서울특별시 도봉구 도봉동,647,도봉파크빌2,84.354,200701,1,1,2002,마들로 835,NaN,...,0,1,127.043376,37.679657,218.273702,2007,01,"252,586.30",4.9,4.5
3,서울특별시 중랑구 상봉동,495,동부,84.990,200701,1,2,1999,신내로7가길 31,NaN,...,0,1,127.095466,37.603957,221.056734,2007,01,"252,586.30",4.9,4.5
4,서울특별시 노원구 중계동,445,염광,59.340,200701,1,8,1996,덕릉로79길 23,NaN,...,0,0,127.076783,37.647301,48.210662,2007,01,"252,586.30",4.9,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 마포구 성산동,446,성산시영,59.430,202306,30,10,1986,월드컵북로 235,NaN,...,0,1,126.903560,37.569738,118.303334,2023,06,"551,569.10",0.9,3.5
1118818,서울특별시 강동구 길동,365-1,라인,59.940,202306,30,10,1998,천중로50길 48-12,NaN,...,0,0,127.146753,37.540088,23.427296,2023,06,"551,569.10",0.9,3.5
1118819,서울특별시 마포구 대흥동,12-41,서희스타힐스,12.850,202306,30,4,2014,신촌로 160,NaN,...,1,0,126.942706,37.556870,72.230221,2023,06,"551,569.10",0.9,3.5
1118820,서울특별시 마포구 대흥동,12-41,서희스타힐스,12.850,202306,30,18,2014,신촌로 160,NaN,...,1,0,126.942706,37.556870,72.230221,2023,06,"551,569.10",0.9,3.5


In [30]:
train

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,화장실개수2,공간개수,유동승객수,역까지_거리,1차역세권,2차역세권,X좌표,Y좌표,정류장까지_거리,is_test
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,NaN,NaN,47694,699.547185,0,0,127.057864,37.476564,73.917793,0
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,NaN,NaN,47694,699.547185,0,0,127.057864,37.476564,73.917793,0
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,NaN,NaN,47694,699.547185,0,0,127.057864,37.476564,73.917793,0
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,NaN,NaN,47694,699.547185,0,0,127.057864,37.476564,73.917793,0
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,NaN,NaN,47694,699.547185,0,0,127.057864,37.476564,73.917793,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 은평구 구산동,382,갈현현대,59.94,200707,12,11,1998,서오릉로21길 36,NaN,...,1.0,3.0,168633,1297.741653,0,0,126.906101,37.612291,68.236190,0
1118818,서울특별시 은평구 구산동,382,갈현현대,59.94,200708,25,10,1998,서오릉로21길 36,NaN,...,1.0,3.0,168633,1297.741653,0,0,126.906101,37.612291,68.236190,0
1118819,서울특별시 은평구 구산동,382,갈현현대,84.83,200708,31,20,1998,서오릉로21길 36,NaN,...,2.0,5.0,168633,1297.741653,0,0,126.906101,37.612291,68.236190,0
1118820,서울특별시 은평구 구산동,382,갈현현대,84.83,200709,15,8,1998,서오릉로21길 36,NaN,...,2.0,5.0,168633,1297.741653,0,0,126.906101,37.612291,68.236190,0


In [31]:
save_test = new_concat[new_concat["is_test"]==1]
del save_test["is_test"]
save_test.to_csv('test_merge_0119_01(cp949).csv', index=False, encoding='cp949')
save_test.to_csv('test_merge_0119_01(UTF-8).csv', index=False, encoding='UTF-8')

In [32]:
save_test

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,1차역세권,2차역세권,X좌표,Y좌표,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리
1118822,서울특별시 성북구 하월곡동,225,래미안월곡,59.7540,202307,1,9,2006,오패산로 90,NaN,...,0,1,127.037620,37.604146,19.799229,2023,07,"563,921.50",1.4,3.5
1118823,서울특별시 강동구 상일동,521,고덕자이,59.9398,202307,1,18,2021,상일로 55,NaN,...,0,0,127.171294,37.550913,100.165657,2023,07,"563,921.50",1.4,3.5
1118824,서울특별시 송파구 가락동,913,헬리오시티,84.9500,202307,1,4,2018,송파대로 345,NaN,...,0,0,127.100695,37.499020,221.599396,2023,07,"563,921.50",1.4,3.5
1118825,서울특별시 강서구 내발산동,759,마곡수명산파크6단지,84.8200,202307,1,13,2007,수명로2길 63,NaN,...,0,0,126.825397,37.551408,84.464625,2023,07,"563,921.50",1.4,3.5
1118826,서울특별시 성북구 길음동,1286,길음뉴타운9단지,59.9800,202307,1,28,2010,길음로9길 50,NaN,...,0,0,127.020186,37.609024,33.004560,2023,07,"563,921.50",1.4,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 성북구 하월곡동,222,월곡두산위브,84.8430,202309,25,13,2003,오패산로 46,20230926.0,...,0,1,127.038892,37.605201,135.628330,2023,09,"563,921.50",1.4,3.5
1128090,서울특별시 도봉구 쌍문동,59,한양2,35.1000,202309,25,11,1988,해등로 231,NaN,...,0,0,127.027833,37.655624,163.035643,2023,09,"563,921.50",1.4,3.5
1128091,서울특별시 영등포구 문래동3가,93-1,문래건영,59.4000,202309,25,11,1998,문래로 94-7,20230926.0,...,1,0,126.896353,37.516949,49.484311,2023,09,"563,921.50",1.4,3.5
1128092,서울특별시 송파구 방이동,50-10,동광팰리스,22.5700,202309,26,14,2018,오금로11길 63-13,NaN,...,0,1,127.119104,37.514828,66.815991,2023,09,"563,921.50",1.4,3.5


In [33]:
test

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,화장실개수2,공간개수,유동승객수,역까지_거리,1차역세권,2차역세권,X좌표,Y좌표,정류장까지_거리,is_test
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.9700,202307,26,5,1987,언주로 3,NaN,...,NaN,NaN,47694,699.547185,0,0,127.057864,37.476564,73.917793,1
1,서울특별시 강남구 개포동,651-1,개포더샵트리에,108.2017,202308,15,10,2021,개포로 311,NaN,...,NaN,NaN,47694,303.229871,0,1,127.057080,37.483713,109.787878,1
2,서울특별시 강남구 개포동,652,개포우성3차,161.0000,202307,28,15,1984,개포로 307,NaN,...,NaN,NaN,47694,374.851509,0,1,127.056890,37.483174,111.101602,1
3,서울특별시 강남구 개포동,652,개포우성3차,133.4600,202308,10,14,1984,개포로 307,NaN,...,NaN,NaN,47694,374.851509,0,1,127.056890,37.483174,111.101602,1
4,서울특별시 강남구 개포동,652,개포우성3차,104.4300,202308,18,6,1984,개포로 307,NaN,...,NaN,NaN,47694,374.851509,0,1,127.056890,37.483174,111.101602,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,신내우디안1단지,84.6500,202307,19,13,2014,신내역로1길 85,20230801.0,...,2.0,5.0,43578,628.586509,0,0,127.107073,37.619326,49.828552,1
9268,서울특별시 중랑구 신내동,816,신내우디안1단지,84.6200,202307,25,12,2014,신내역로1길 85,NaN,...,2.0,5.0,43578,628.586509,0,0,127.107073,37.619326,49.828552,1
9269,서울특별시 중랑구 신내동,816,신내우디안1단지,101.6500,202308,27,12,2014,신내역로1길 85,NaN,...,2.0,6.0,43578,628.586509,0,0,127.107073,37.619326,49.828552,1
9270,서울특별시 중랑구 신내동,816,신내우디안1단지,84.9400,202309,2,18,2014,신내역로1길 85,NaN,...,2.0,5.0,43578,628.586509,0,0,127.107073,37.619326,49.828552,1


## 학군 병합

In [144]:
# 데이터 업데이트
train_path = '../../data/train_merge_0119_01(UTF-8).csv'
test_path  = '../../data/test_merge_0119_01(UTF-8).csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [145]:
# 학군 데이터
edu_group_path = '../../data/education_group.csv'
edu_group = pd.read_csv(edu_group_path)

In [146]:
edu_group

,구,학군
0,동대문구,1
1,중랑구,1
2,마포구,2
3,서대문구,2
4,은평구,2
5,구로구,3
6,금천구,3
7,영등포구,3
8,노원구,4
9,도봉구,4


In [147]:
concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,2차역세권,X좌표,Y좌표,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,is_test
0,서울특별시 강서구 마곡동,429,벽산,59.980,200701,1,15,1999,양천로47길 118,NaN,...,0,126.825674,37.566753,41.796893,2007,1,"252,586.30",4.9,4.5,0
1,서울특별시 노원구 월계동,322-1,성원,59.940,200701,1,9,1995,마들로3길 37,NaN,...,1,127.061686,37.631583,192.711284,2007,1,"252,586.30",4.9,4.5,0
2,서울특별시 도봉구 도봉동,647,도봉파크빌2,84.354,200701,1,1,2002,마들로 835,NaN,...,1,127.043376,37.679657,218.273702,2007,1,"252,586.30",4.9,4.5,0
3,서울특별시 중랑구 상봉동,495,동부,84.990,200701,1,2,1999,신내로7가길 31,NaN,...,1,127.095466,37.603957,221.056734,2007,1,"252,586.30",4.9,4.5,0
4,서울특별시 노원구 중계동,445,염광,59.340,200701,1,8,1996,덕릉로79길 23,NaN,...,0,127.076783,37.647301,48.210662,2007,1,"252,586.30",4.9,4.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 성북구 하월곡동,222,월곡두산위브,84.843,202309,25,13,2003,오패산로 46,20230926.0,...,1,127.038892,37.605201,135.628330,2023,9,"563,921.50",1.4,3.5,1
9268,서울특별시 도봉구 쌍문동,59,한양2,35.100,202309,25,11,1988,해등로 231,NaN,...,0,127.027833,37.655624,163.035643,2023,9,"563,921.50",1.4,3.5,1
9269,서울특별시 영등포구 문래동3가,93-1,문래건영,59.400,202309,25,11,1998,문래로 94-7,20230926.0,...,0,126.896353,37.516949,49.484311,2023,9,"563,921.50",1.4,3.5,1
9270,서울특별시 송파구 방이동,50-10,동광팰리스,22.570,202309,26,14,2018,오금로11길 63-13,NaN,...,1,127.119104,37.514828,66.815991,2023,9,"563,921.50",1.4,3.5,1


In [148]:
new_concat = pd.merge(concat, edu_group, on="구", how='inner')

In [149]:
new_concat[["구", "학군"]]

,구,학군
0,강서구,7
1,강서구,7
2,강서구,7
3,강서구,7
4,강서구,7
...,...,...
1128089,광진구,10
1128090,광진구,10
1128091,광진구,10
1128092,광진구,10


In [150]:
new_concat.columns

Index(['시군구', '번지', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)',
       'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y', '단지신청일', 'target', '시', '구', '동',
       '지번주소', '동아파트명', '전용면적2', '방개수2', '화장실개수2', '공간개수', '유동승객수', '역까지_거리',
       '1차역세권', '2차역세권', 'X좌표', 'Y좌표', '정류장까지_거리', '계약년', '계약월',
       '국내총생산(명목GDP)', '경제성장률(실질GDP성장률)', '기준금리', 'is_test', '학군'],
      dtype='object')

## 전세가율 병합

In [151]:
# 전세가율 데이터
lease_rate_path = '../../data/lease_rate.csv'
lease_rate = pd.read_csv(lease_rate_path, encoding='cp949')

In [167]:
lease_rate.rename(columns={"금리":"전세가율"}, inplace=True)

In [168]:
lease_rate

,계약년월,전세가율,구
0,201304,50.14,강남구
1,201305,50.17,강남구
2,201306,50.61,강남구
3,201307,51.41,강남구
4,201308,52.07,강남구
...,...,...,...
3145,202305,58.39,중랑구
3146,202306,58.59,중랑구
3147,202307,58.38,중랑구
3148,202308,58.48,중랑구


In [169]:
new_concat.columns

Index(['시군구', '번지', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)',
       'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y', '단지신청일', 'target', '시', '구', '동',
       '지번주소', '동아파트명', '전용면적2', '방개수2', '화장실개수2', '공간개수', '유동승객수', '역까지_거리',
       '1차역세권', '2차역세권', 'X좌표', 'Y좌표', '정류장까지_거리', '계약년', '계약월',
       '국내총생산(명목GDP)', '경제성장률(실질GDP성장률)', '기준금리', 'is_test', '학군'],
      dtype='object')

In [172]:
new_concat = pd.merge(new_concat, lease_rate, on=['구', '계약년월'], how='left')

In [173]:
new_concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,Y좌표,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,is_test,학군,전세가율
0,서울특별시 강서구 마곡동,429,벽산,59.98,200701,1,15,1999,양천로47길 118,NaN,...,37.566753,41.796893,2007,1,"252,586.30",4.9,4.5,0,7,NaN
1,서울특별시 강서구 방화동,832,청솔,49.20,200701,2,2,1994,방화대로47길 55,NaN,...,37.576902,79.468057,2007,1,"252,586.30",4.9,4.5,0,7,NaN
2,서울특별시 강서구 방화동,814,방화5,33.18,200701,2,13,1994,금낭화로 287-19,NaN,...,37.576902,79.468057,2007,1,"252,586.30",4.9,4.5,0,7,NaN
3,서울특별시 강서구 화곡동,1139,초록,59.34,200701,2,10,1998,강서로45다길 30-27,NaN,...,37.543428,92.876958,2007,1,"252,586.30",4.9,4.5,0,7,NaN
4,서울특별시 강서구 화곡동,927-1,웰라이빌,84.98,200701,2,7,2000,강서로5나길 104,NaN,...,37.543428,92.876958,2007,1,"252,586.30",4.9,4.5,0,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 광진구 구의동,631-1,현대프라임,59.82,202309,14,21,1997,광나루로56길 29,NaN,...,37.543393,83.793367,2023,9,"563,921.50",1.4,3.5,1,10,51.2
1128090,서울특별시 광진구 자양동,636-11,그랜드,48.28,202309,15,6,2023,뚝섬로52길 7,NaN,...,37.533947,98.685962,2023,9,"563,921.50",1.4,3.5,1,10,51.2
1128091,서울특별시 광진구 광장동,484,현대3,59.67,202309,15,8,1990,아차산로70길 62,NaN,...,37.547630,115.740882,2023,9,"563,921.50",1.4,3.5,1,10,51.2
1128092,서울특별시 광진구 구의동,199-18,삼성쉐르빌,84.94,202309,16,17,2001,구의강변로 106,NaN,...,37.543393,83.793367,2023,9,"563,921.50",1.4,3.5,1,10,51.2


## 재개발 개수 병합

In [174]:
redevelopment_path = '../../data/redevelopment_num.csv'
redevelopment = pd.read_csv(redevelopment_path)

In [176]:
redevelopment.rename(columns={"연도" : "계약년"}, inplace=True)

In [177]:
redevelopment

,구,계약년,재개발개수
0,강남구,2007,0
1,강남구,2008,0
2,강남구,2009,0
3,강남구,2010,0
4,강남구,2011,0
...,...,...,...
420,관악구,2023,0
421,서초구,2023,0
422,강남구,2023,0
423,송파구,2023,0


In [180]:
new_concat = pd.merge(new_concat, redevelopment, on=['구', '계약년'], how='left')

In [184]:
new_concat

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,is_test,학군,전세가율,재개발개수
0,서울특별시 강서구 마곡동,429,벽산,59.98,200701,1,15,1999,양천로47길 118,NaN,...,41.796893,2007,1,"252,586.30",4.9,4.5,0,7,NaN,0
1,서울특별시 강서구 방화동,832,청솔,49.20,200701,2,2,1994,방화대로47길 55,NaN,...,79.468057,2007,1,"252,586.30",4.9,4.5,0,7,NaN,0
2,서울특별시 강서구 방화동,814,방화5,33.18,200701,2,13,1994,금낭화로 287-19,NaN,...,79.468057,2007,1,"252,586.30",4.9,4.5,0,7,NaN,0
3,서울특별시 강서구 화곡동,1139,초록,59.34,200701,2,10,1998,강서로45다길 30-27,NaN,...,92.876958,2007,1,"252,586.30",4.9,4.5,0,7,NaN,0
4,서울특별시 강서구 화곡동,927-1,웰라이빌,84.98,200701,2,7,2000,강서로5나길 104,NaN,...,92.876958,2007,1,"252,586.30",4.9,4.5,0,7,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 광진구 구의동,631-1,현대프라임,59.82,202309,14,21,1997,광나루로56길 29,NaN,...,83.793367,2023,9,"563,921.50",1.4,3.5,1,10,51.2,0
1128090,서울특별시 광진구 자양동,636-11,그랜드,48.28,202309,15,6,2023,뚝섬로52길 7,NaN,...,98.685962,2023,9,"563,921.50",1.4,3.5,1,10,51.2,0
1128091,서울특별시 광진구 광장동,484,현대3,59.67,202309,15,8,1990,아차산로70길 62,NaN,...,115.740882,2023,9,"563,921.50",1.4,3.5,1,10,51.2,0
1128092,서울특별시 광진구 구의동,199-18,삼성쉐르빌,84.94,202309,16,17,2001,구의강변로 106,NaN,...,83.793367,2023,9,"563,921.50",1.4,3.5,1,10,51.2,0


In [181]:
new_concat["재개발개수"].isna().sum()

0

In [183]:
new_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 1128093
Data columns (total 54 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1128094 non-null  object 
 1   번지                      1128094 non-null  object 
 2   아파트명                    1125958 non-null  object 
 3   전용면적(㎡)                 1128094 non-null  float64
 4   계약년월                    1128094 non-null  int64  
 5   계약일                     1128094 non-null  int64  
 6   층                       1128094 non-null  int64  
 7   건축년도                    1128094 non-null  int64  
 8   도로명                     1128094 non-null  object 
 9   해제사유발생일                 6195 non-null     float64
 10  거래유형                    41643 non-null    object 
 11  중개사소재지                  38513 non-null    object 
 12  k-단지분류(아파트,주상복합등등)      250821 non-null   object 
 13  k-세대타입(분양형태)            251969 non-null   object 
 14  k-

In [185]:
save_train = new_concat[new_concat["is_test"]==0]
del save_train["is_test"]
save_train.to_csv('train_merge_0119_02(cp949).csv', index=False, encoding='cp949')
save_train.to_csv('train_merge_0119_02(UTF-8).csv', index=False, encoding='UTF-8')

In [191]:
save_test = new_concat[new_concat["is_test"]==1]
del save_test["is_test"]
save_test.to_csv('test_merge_0119_02(cp949).csv', index=False, encoding='cp949')
save_test.to_csv('test_merge_0119_02(UTF-8).csv', index=False, encoding='UTF-8')

In [188]:
save_train

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,Y좌표,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,학군,전세가율,재개발개수
0,서울특별시 강서구 마곡동,429,벽산,59.98,200701,1,15,1999,양천로47길 118,NaN,...,37.566753,41.796893,2007,1,"252,586.30",4.9,4.5,7,NaN,0
1,서울특별시 강서구 방화동,832,청솔,49.20,200701,2,2,1994,방화대로47길 55,NaN,...,37.576902,79.468057,2007,1,"252,586.30",4.9,4.5,7,NaN,0
2,서울특별시 강서구 방화동,814,방화5,33.18,200701,2,13,1994,금낭화로 287-19,NaN,...,37.576902,79.468057,2007,1,"252,586.30",4.9,4.5,7,NaN,0
3,서울특별시 강서구 화곡동,1139,초록,59.34,200701,2,10,1998,강서로45다길 30-27,NaN,...,37.543428,92.876958,2007,1,"252,586.30",4.9,4.5,7,NaN,0
4,서울특별시 강서구 화곡동,927-1,웰라이빌,84.98,200701,2,7,2000,강서로5나길 104,NaN,...,37.543428,92.876958,2007,1,"252,586.30",4.9,4.5,7,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127907,서울특별시 광진구 자양동,761,자양2현대,59.95,202306,28,1,1995,뚝섬로34길 9,NaN,...,37.533947,98.685962,2023,6,"551,569.10",0.9,3.5,10,50.83,0
1127908,서울특별시 광진구 자양동,762,한라,59.96,202306,29,8,1996,뚝섬로35길 10,20230721.0,...,37.533947,98.685962,2023,6,"551,569.10",0.9,3.5,10,50.83,0
1127909,서울특별시 광진구 자양동,762,한라,59.96,202306,29,8,1996,뚝섬로35길 10,NaN,...,37.533947,98.685962,2023,6,"551,569.10",0.9,3.5,10,50.83,0
1127910,서울특별시 광진구 광장동,218-1,극동2,75.55,202306,30,8,1989,아차산로 552,NaN,...,37.547630,115.740882,2023,6,"551,569.10",0.9,3.5,10,50.83,0


In [190]:
save_test

,시군구,번지,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,Y좌표,정류장까지_거리,계약년,계약월,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,학군,전세가율,재개발개수
66610,서울특별시 강서구 내발산동,759,마곡수명산파크6단지,84.8200,202307,1,13,2007,수명로2길 63,NaN,...,37.551408,84.464625,2023,7,"563,921.50",1.4,3.5,7,51.77,0
66611,서울특별시 강서구 마곡동,748,마곡13단지힐스테이트마스터,59.9825,202307,1,13,2017,마곡중앙1로 71,NaN,...,37.566753,41.796893,2023,7,"563,921.50",1.4,3.5,7,51.77,0
66612,서울특별시 강서구 가양동,1475,강변,39.6000,202307,1,3,1992,허준로 139,NaN,...,37.565588,89.618802,2023,7,"563,921.50",1.4,3.5,7,51.77,0
66613,서울특별시 강서구 염창동,262-2,대림,59.9400,202307,1,10,1999,공항대로75길 42,NaN,...,37.554659,69.179580,2023,7,"563,921.50",1.4,3.5,7,51.77,0
66614,서울특별시 강서구 염창동,262-2,대림,59.9400,202307,1,11,1999,공항대로75길 42,NaN,...,37.554659,69.179580,2023,7,"563,921.50",1.4,3.5,7,51.77,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 광진구 구의동,631-1,현대프라임,59.8200,202309,14,21,1997,광나루로56길 29,NaN,...,37.543393,83.793367,2023,9,"563,921.50",1.4,3.5,10,51.20,0
1128090,서울특별시 광진구 자양동,636-11,그랜드,48.2800,202309,15,6,2023,뚝섬로52길 7,NaN,...,37.533947,98.685962,2023,9,"563,921.50",1.4,3.5,10,51.20,0
1128091,서울특별시 광진구 광장동,484,현대3,59.6700,202309,15,8,1990,아차산로70길 62,NaN,...,37.547630,115.740882,2023,9,"563,921.50",1.4,3.5,10,51.20,0
1128092,서울특별시 광진구 구의동,199-18,삼성쉐르빌,84.9400,202309,16,17,2001,구의강변로 106,NaN,...,37.543393,83.793367,2023,9,"563,921.50",1.4,3.5,10,51.20,0


## 숲세권 정보 병합

In [3]:
# 데이터 업데이트
train_path = '../../data/KOREAN_train4.csv'
test_path  = '../../data/KOREAN_test4.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [4]:
forest_range_score_path = '../../data/forest_range_score.csv'
index_path = '../../data/forest_range_score_categori.csv'
forest_range_score = pd.read_csv(forest_range_score_path)
index = pd.read_csv(index_path, encoding='cp949')

In [5]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LGDNG_ADDR,법정동주소,VARCHAR(100)
5,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LILHD_NDX,생활지수,NUMBER(24)
6,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LILHD_NDX_NM,생활지수명,VARCHAR(10)
7,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,EVSCR,평가점수,"NUMBER(5,2)"


In [6]:
forest_range_score

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,lgdng_addr,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,2,중상위권,76
1,11110,1111011800,A11007001,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,2,중상위권,84
2,11110,1111011800,A11087101,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,2,중상위권,84
3,11110,1111012400,A11014001,로얄팰리스스위트,서울특별시 종로구 수송동 85 로얄팰리스스위트,2,중상위권,77
4,11110,1111013300,A11034001,현대뜨레비앙,서울특별시 종로구 익선동 55 현대뜨레비앙,1,상위권,86
...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,서울특별시 강동구 강일동 673 강일리버파크4단지,4,중하위권,63
2639,11740,1174011000,A13410004,강일리버파크6단지,서울특별시 강동구 강일동 684 강일리버파크6단지,3,중위권,73
2640,11740,1174011000,A13410010,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,3,중위권,64
2641,11740,1174011000,A13410009,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,4,중하위권,63


In [7]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [8]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'lgdng_addr': '법정동주소',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [9]:
variables = {'sgng_cd': '시군구코드',
             'lgdng_cd': '법정동코드',
             'hscmp_cd': '단지코드',
             'hscmp_nm': '단지명',
             'lgdng_addr': '법정동주소',
             'lilhd_ndx': '숲세권생활지수',
             'lilhd_ndx_nm': '숲세권생활지수명',
             'evscr': '숲세권평가점수'}

In [10]:
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'lgdng_addr': '법정동주소',
 'lilhd_ndx': '숲세권생활지수',
 'lilhd_ndx_nm': '숲세권생활지수명',
 'evscr': '숲세권평가점수'}

In [11]:
forest_range_score.rename(columns=variables, inplace=True) # column의 이름 변경

In [12]:
use_forest_range_score = forest_range_score.drop(columns=["시군구코드", "법정동코드", "단지코드"], axis=1)

In [13]:
def remove_substring(df):
    return df["법정동주소"].replace(df["단지명"], "")    

In [14]:
use_forest_range_score["법정동주소"] = use_forest_range_score.apply(remove_substring, axis=1)

In [15]:
del use_forest_range_score["단지명"]

In [16]:
# 서울특별시 종로구 사직동 9, 서울특별시 종로구 익선동 55, 서울특별시 강동구 강일동 689
use_forest_range_score["법정동주소"] = use_forest_range_score["법정동주소"].str.strip()

In [17]:
use_forest_range_score = use_forest_range_score[~use_forest_range_score["법정동주소"].duplicated()]

In [18]:
concat["법정동주소"] = concat["시군구"] + " " + concat["번지"].astype(str)

In [19]:
new_concat = pd.merge(concat, use_forest_range_score, on=["법정동주소"], how='left')

In [20]:
new_concat

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,명목gdp,학군,재개발개수,학급수,여당득표율,is_test,법정동주소,숲세권생활지수,숲세권생활지수명,숲세권평가점수
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,"449,357.80",8,0,30.781250,35.36,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,"449,357.80",8,0,30.781250,35.36,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0
1128090,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0
1128091,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0
1128092,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0


In [21]:
concat

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,전체계약일자,금리,실질gdp,명목gdp,학군,재개발개수,학급수,여당득표율,is_test,법정동주소
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,20171208,1.5,2.9,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,20171222,1.5,2.9,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,20171228,1.5,2.9,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,20180103,1.5,2.9,"449,357.80",8,0,30.781250,35.36,0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,20180108,1.5,2.9,"449,357.80",8,0,30.781250,35.36,0,서울특별시 강남구 개포동 658-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,20230719,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816
9268,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,20230725,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816
9269,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,20230827,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816
9270,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,20230902,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816


## 지천 생활권 정보 병합


In [22]:
# 지천 생활권 정보
river_range_path = '../../data/river_stamp.csv'
index_path = '../../data/river_stamp_categori.csv'
river_range = pd.read_csv(river_range_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [23]:
river_range

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,nmhsh,lgdng_addr,cours_cnt,cours_tot_lngth,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,0,0.00,5,하위권,0
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,1,8179.50,4,중하위권,84
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,1,8179.50,4,중하위권,84
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,1,8179.50,4,중하위권,84
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,2,8064.31,4,중하위권,84
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,2,8064.31,4,중하위권,84
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,2,8064.31,4,중하위권,84
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,2,8064.31,4,중하위권,84


In [24]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,NMHSH,세대수,NUMBER(20)
5,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LGDNG_ADDR,법정동주소,VARCHAR(100)
6,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,COURS_CNT,코스수,NUMBER(14)
7,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,COURS_TOT_LNGTH,코스총길이,"NUMBER(10,2)"
8,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LILHD_NDX,생활지수,NUMBER(24)
9,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LILHD_NDX_NM,생활지수명,VARCHAR(10)


In [25]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [26]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '코스수',
 'cours_tot_lngth': '코스총길이',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [27]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '지천코스수',
 'cours_tot_lngth': '지천코스총길이',
 'lilhd_ndx': '지천생활지수',
 'lilhd_ndx_nm': '지천생활지수명',
 'evscr': '지천평가점수'}

In [28]:
river_range.rename(columns=variables, inplace=True) # column의 이름 변경

In [29]:
river_range

,시군구코드,법정동코드,단지코드,단지명,세대수,법정동주소,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,0,0.00,5,하위권,0
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,1,8179.50,4,중하위권,84
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,1,8179.50,4,중하위권,84
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,1,8179.50,4,중하위권,84
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,2,8064.31,4,중하위권,84
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,2,8064.31,4,중하위권,84
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,2,8064.31,4,중하위권,84
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,2,8064.31,4,중하위권,84


In [30]:
use_river_range = river_range.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [31]:
use_river_range

,단지명,법정동주소,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,0,0.00,5,하위권,0
1,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,1,8179.50,4,중하위권,84
2,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,1,8179.50,4,중하위권,84
3,로얄팰리스스위트,서울특별시 종로구 수송동 85 로얄팰리스스위트,1,8179.50,4,중하위권,84
4,현대뜨레비앙,서울특별시 종로구 익선동 55 현대뜨레비앙,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673 강일리버파크4단지,2,8064.31,4,중하위권,84
2639,강일리버파크6단지,서울특별시 강동구 강일동 684 강일리버파크6단지,2,8064.31,4,중하위권,84
2640,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,2,8064.31,4,중하위권,84
2641,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,2,8064.31,4,중하위권,84


In [32]:
use_river_range["법정동주소"] = use_river_range.apply(remove_substring, axis=1)

In [33]:
# 서울특별시 종로구 사직동 9, 서울특별시 종로구 익선동 55, 서울특별시 강동구 강일동 689
use_river_range["법정동주소"] = use_river_range["법정동주소"].str.strip()

In [34]:
use_river_range = use_river_range[~use_river_range["법정동주소"].duplicated()]

In [35]:
use_river_range

,단지명,법정동주소,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9,0,0.00,5,하위권,0
1,경희궁의아침3단지,서울특별시 종로구 내수동 72,1,8179.50,4,중하위권,84
2,경희궁의아침2단지,서울특별시 종로구 내수동 71,1,8179.50,4,중하위권,84
3,로얄팰리스스위트,서울특별시 종로구 수송동 85,1,8179.50,4,중하위권,84
4,현대뜨레비앙,서울특별시 종로구 익선동 55,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673,2,8064.31,4,중하위권,84
2639,강일리버파크6단지,서울특별시 강동구 강일동 684,2,8064.31,4,중하위권,84
2640,강일리버파크7단지,서울특별시 강동구 강일동 686,2,8064.31,4,중하위권,84
2641,강일리버파크5단지,서울특별시 강동구 강일동 670,2,8064.31,4,중하위권,84


In [36]:
del use_river_range["단지명"]

In [37]:
new_concat = pd.merge(new_concat, use_river_range, on=["법정동주소"], how='left')

In [38]:
new_concat

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,is_test,법정동주소,숲세권생활지수,숲세권생활지수명,숲세권평가점수,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,0,서울특별시 강남구 개포동 658-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0,1.0,3035.02,4.0,중하위권,74.0
1128090,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0,1.0,3035.02,4.0,중하위권,74.0
1128091,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0,1.0,3035.02,4.0,중하위권,74.0
1128092,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,1,서울특별시 중랑구 신내동 816,4.0,중하위권,54.0,1.0,3035.02,4.0,중하위권,74.0


## 도시공원 생활권 정보 병합

In [39]:
# 도시공원 생활권 정보
city_park_path = '../../data/city_park.csv'
index_path = '../../data/city_park_categori.csv'
city_park = pd.read_csv(city_park_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [40]:
city_park

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,nmhsh,lgdng_addr,park_cnt,park_ttar,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,12,317184.27,4,중하위권,81
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,16,321615.17,4,중하위권,81
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,16,322286.17,4,중하위권,81
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,15,67828.60,4,중하위권,71
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,8,533213.30,3,중위권,85
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,7,176616.90,4,중하위권,78
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,10,309300.10,4,중하위권,81
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,9,209700.20,4,중하위권,79
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,7,176616.90,4,중하위권,78


In [41]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,NMHSH,세대수,NUMBER(20)
5,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LGDNG_ADDR,법정동주소,VARCHAR(100)
6,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,PARK_CNT,공원수,NUMBER(14)
7,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,PARK_TTAR,공원총면적,"NUMBER(13,4)"
8,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LILHD_NDX,생활지수,NUMBER(24)
9,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LILHD_NDX_NM,생활지수명,VARCHAR(10)


In [42]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [43]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'park_cnt': '공원수',
 'park_ttar': '공원총면적',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [44]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'park_cnt': '공원수',
 'park_ttar': '공원총면적',
 'lilhd_ndx': '도시공원생활지수',
 'lilhd_ndx_nm': '도시공원생활지수명',
 'evscr': '도시공원평가점수'}

In [45]:
city_park.rename(columns=variables, inplace=True) # column의 이름 변경

In [46]:
use_city_park = city_park.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [47]:
use_city_park["법정동주소"] = use_city_park.apply(remove_substring, axis=1)
use_city_park["법정동주소"] = use_city_park["법정동주소"].str.strip()
use_city_park = use_city_park[~use_city_park["법정동주소"].duplicated()]

In [48]:
del use_city_park["단지명"]

In [49]:
new_concat = pd.merge(new_concat, use_city_park, on=["법정동주소"], how='left')

In [50]:
concat

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,전체계약일자,금리,실질gdp,명목gdp,학군,재개발개수,학급수,여당득표율,is_test,법정동주소
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,20171208,1.5,2.9,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,20171222,1.5,2.9,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,20171228,1.5,2.9,"474,957.90",8,0,29.393939,35.36,0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,20180103,1.5,2.9,"449,357.80",8,0,30.781250,35.36,0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,20180108,1.5,2.9,"449,357.80",8,0,30.781250,35.36,0,서울특별시 강남구 개포동 658-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,20230719,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816
9268,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,20230725,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816
9269,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,20230827,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816
9270,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,20230902,3.5,1.4,"563,921.50",1,0,27.250000,-45.73,1,서울특별시 중랑구 신내동 816


In [51]:
new_concat

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수,공원수,공원총면적,도시공원생활지수,도시공원생활지수명,도시공원평가점수
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,1.0,3035.02,4.0,중하위권,74.0,2.0,27003.0,4.0,중하위권,66.0
1128090,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,1.0,3035.02,4.0,중하위권,74.0,2.0,27003.0,4.0,중하위권,66.0
1128091,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,1.0,3035.02,4.0,중하위권,74.0,2.0,27003.0,4.0,중하위권,66.0
1128092,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,1.0,3035.02,4.0,중하위권,74.0,2.0,27003.0,4.0,중하위권,66.0


## 산책 생활권 정보

In [52]:
# 도시공원 생활권 정보
walking_path = '../../data/walking.csv'
index_path = '../../data/walking_categori.csv'
walking = pd.read_csv(walking_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [53]:
walking

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,nmhsh,lgdng_addr,cours_cnt,cours_tot_lngth,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,9,16916.61,1,상위권,95
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,8,15662.61,1,상위권,94
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,8,15662.61,1,상위권,94
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,6,12605.88,2,중상위권,92
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,4,11385.21,2,중상위권,91
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,0,0.00,5,하위권,0
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,2,3236.52,4,중하위권,79
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,0,0.00,5,하위권,0
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,0,0.00,5,하위권,0


In [54]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,NMHSH,세대수,NUMBER(20)
5,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LGDNG_ADDR,법정동주소,VARCHAR(100)
6,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,COURS_CNT,코스수,NUMBER(14)
7,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,COURS_TOT_LNGTH,코스총길이,"NUMBER(10,2)"
8,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LILHD_NDX,생활지수,NUMBER(24)
9,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LILHD_NDX_NM,생활지수명,VARCHAR(10)


In [55]:
new_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 1128093
Data columns (total 68 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   시군구                1128094 non-null  object 
 1   번지                 1128094 non-null  object 
 2   아파트명               1125958 non-null  object 
 3   전용면적               1128094 non-null  float64
 4   계약년월               1128094 non-null  int64  
 5   계약일                1128094 non-null  int64  
 6   층                  1128094 non-null  int64  
 7   건축년도               1128094 non-null  int64  
 8   도로명                1128094 non-null  object 
 9   해제사유발생일            6195 non-null     float64
 10  거래유형               41643 non-null    object 
 11  중개사소재지             38513 non-null    object 
 12  k-단지분류             250821 non-null   object 
 13  k-세대타입(분양형태)       251969 non-null   object 
 14  k-복도유형             251640 non-null   object 
 15  k-난방방식             251969 non-nu

In [56]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [57]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '코스수',
 'cours_tot_lngth': '코스총길이',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [58]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '산책코스수',
 'cours_tot_lngth': '산책코스총길이',
 'lilhd_ndx': '산책생활지수',
 'lilhd_ndx_nm': '산책생활지수명',
 'evscr': '산책평가점수'}

In [59]:
walking.rename(columns=variables, inplace=True) # column의 이름 변경

In [60]:
use_walking = walking.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [61]:
use_walking

,단지명,법정동주소,산책코스수,산책코스총길이,산책생활지수,산책생활지수명,산책평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,9,16916.61,1,상위권,95
1,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,8,15662.61,1,상위권,94
2,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,8,15662.61,1,상위권,94
3,로얄팰리스스위트,서울특별시 종로구 수송동 85 로얄팰리스스위트,6,12605.88,2,중상위권,92
4,현대뜨레비앙,서울특별시 종로구 익선동 55 현대뜨레비앙,4,11385.21,2,중상위권,91
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673 강일리버파크4단지,0,0.00,5,하위권,0
2639,강일리버파크6단지,서울특별시 강동구 강일동 684 강일리버파크6단지,2,3236.52,4,중하위권,79
2640,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,0,0.00,5,하위권,0
2641,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,0,0.00,5,하위권,0


In [62]:
use_walking["법정동주소"] = use_walking.apply(remove_substring, axis=1)
use_walking["법정동주소"] = use_walking["법정동주소"].str.strip()
use_walking = use_walking[~use_walking["법정동주소"].duplicated()]

In [63]:
use_walking

,단지명,법정동주소,산책코스수,산책코스총길이,산책생활지수,산책생활지수명,산책평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9,9,16916.61,1,상위권,95
1,경희궁의아침3단지,서울특별시 종로구 내수동 72,8,15662.61,1,상위권,94
2,경희궁의아침2단지,서울특별시 종로구 내수동 71,8,15662.61,1,상위권,94
3,로얄팰리스스위트,서울특별시 종로구 수송동 85,6,12605.88,2,중상위권,92
4,현대뜨레비앙,서울특별시 종로구 익선동 55,4,11385.21,2,중상위권,91
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673,0,0.00,5,하위권,0
2639,강일리버파크6단지,서울특별시 강동구 강일동 684,2,3236.52,4,중하위권,79
2640,강일리버파크7단지,서울특별시 강동구 강일동 686,0,0.00,5,하위권,0
2641,강일리버파크5단지,서울특별시 강동구 강일동 670,0,0.00,5,하위권,0


In [64]:
del use_walking["단지명"]

In [68]:
new_concat = pd.merge(new_concat, use_walking, on=["법정동주소"], how='left')

## 생태공원 정보 병합

In [70]:
# 생태공원 정보
ecology_path = '../../data/ecology.csv'
index_path = '../../data/ecology_categori.csv'
ecology = pd.read_csv(ecology_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [71]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [72]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '코스수',
 'cours_tot_lngth': '코스총길이',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [73]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '생태공원코스수',
 'cours_tot_lngth': '생태공원코스총길이',
 'lilhd_ndx': '생태공원생활지수',
 'lilhd_ndx_nm': '생태공원생활지수명',
 'evscr': '생태공원평가점수'}

In [74]:
ecology.rename(columns=variables, inplace=True) # column의 이름 변경

In [75]:
use_ecology = ecology.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [76]:
use_ecology["법정동주소"] = use_ecology.apply(remove_substring, axis=1)
use_ecology["법정동주소"] = use_ecology["법정동주소"].str.strip()
use_ecology = use_ecology[~use_ecology["법정동주소"].duplicated()]

In [78]:
del use_ecology["단지명"]

In [80]:
new_concat = pd.merge(new_concat, use_ecology, on=["법정동주소"], how='left')

In [81]:
new_concat.columns

Index(['시군구', '번지', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류', 'k-세대타입(분양형태)', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사', 'k-시행사', 'k-연면적', 'k-주거전용면적',
       'k-전용면적별세대현황60이하', 'k-전용면적별세대현황6085이하', 'k-85135이하', 'k-135초과',
       '세대전기계약방법', '건축면적', '주차대수', '기타의무임대1234', '단지신청일', 'target', '구', '동',
       '전체주소명', '동아파트명', '유동인구', '아파트 지하철역 거리', '1차역세권', '2차역세권',
       '아파트 버스정류장 거리', '좌표X', '좌표Y', '아파트 한강대교 거리', '전세가율', '전체계약일자', '금리',
       '실질gdp', '명목gdp', '학군', '재개발개수', '학급수', '여당득표율', 'is_test', '법정동주소',
       '숲세권생활지수', '숲세권생활지수명', '숲세권평가점수', '지천코스수', '지천코스총길이', '지천생활지수',
       '지천생활지수명', '지천평가점수', '공원수', '공원총면적', '도시공원생활지수', '도시공원생활지수명',
       '도시공원평가점수', '산책코스수', '산책코스총길이', '산책생활지수', '산책생활지수명', '산책평가점수',
       '생태공원코스수', '생태공원코스총길이', '생태공원생활지수', '생태공원생활지수명', '생태공원평가점수'],
      dtype='object')

In [82]:
new_concat

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,산책코스수,산책코스총길이,산책생활지수,산책생활지수명,산책평가점수,생태공원코스수,생태공원코스총길이,생태공원생활지수,생태공원생활지수명,생태공원평가점수
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,신내우디안1단지,84.65,202307,19,13,2014,신내역로1길 85,20230801.0,...,0.0,0.0,5.0,하위권,0.0,0.0,0.0,5.0,하위권,0.0
1128090,서울특별시 중랑구 신내동,816,신내우디안1단지,84.62,202307,25,12,2014,신내역로1길 85,NaN,...,0.0,0.0,5.0,하위권,0.0,0.0,0.0,5.0,하위권,0.0
1128091,서울특별시 중랑구 신내동,816,신내우디안1단지,101.65,202308,27,12,2014,신내역로1길 85,NaN,...,0.0,0.0,5.0,하위권,0.0,0.0,0.0,5.0,하위권,0.0
1128092,서울특별시 중랑구 신내동,816,신내우디안1단지,84.94,202309,2,18,2014,신내역로1길 85,NaN,...,0.0,0.0,5.0,하위권,0.0,0.0,0.0,5.0,하위권,0.0


In [83]:
save_train = new_concat[new_concat["is_test"]==0]
del save_train["is_test"]
save_train.to_csv('train_merge_0122_01(cp949)_final.csv', index=False, encoding='cp949')
save_train.to_csv('train_merge_0122_01(UTF-8)_final.csv', index=False, encoding='UTF-8')

In [84]:
save_test = new_concat[new_concat["is_test"]==1]
del save_test["is_test"]
save_test.to_csv('test_merge_0122_01(cp949)_final.csv', index=False, encoding='cp949')
save_test.to_csv('test_merge_0122_01(UTF-8)_final.csv', index=False, encoding='UTF-8')